<a href="https://colab.research.google.com/github/harsharma0902/Sentiment-Analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SENTIMENTAL ANALYSIS

The INTERNET today is full of social networking websites such as Facebook, LinkedIn, Pinterest, etc. which contain a lot of data. For example, Twitter is a microblogging site in which users can post updates (Tweets) to friends (followers). It has become an immense dataset of the so-called SENTIMENTS. 

In this project, I analyzed various papers focusing on the topic of Sentiment Analysis using Machine Learning. This Sentiment Analysis Model based on Naive Bayes & Support Vector Machine. Its purpose is to analyze sentiment more effectively. This model proved to be highly effective and accurate on the analysis of feelings.

In [1]:
#DESCRIPTION: SENTIMENTAL ANALYSIS OF TWITTER DATA
import tweepy
import json
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
#TWITTER API CREDENTIALS
consumerKey = "FLj6bcrpPsic3rZ6kLQNa7mbH"
consumerSecret = "psOlDGnwnqBd6xo20mVHlGkAGvvKt7C6OcsysJphXQnajOrcD4"
accessToken = "1442491980554981384-v9ThsGb9jnogNgzCM0kgPJjxvKQPGb"
accessTokenSecret = "DwXNMU82ievaZRw9mh4wq0AnzIYZRkHOwmGlhmmxvLjGK"

In [3]:
#CREATING AUTHENTICATION OBJECT
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)

#SET ACCESS TOKEN AND ACCESS TOKEN SECRET
authenticate.set_access_token(accessToken, accessTokenSecret)

#CREATING API OBJECT WHILE PASSING AUTH INFORMATION
T_API = tweepy.API(authenticate, wait_on_rate_limit=True)

In [4]:
Tweet_List=[]
class MyStreamListener(tweepy.StreamListener):
    def __init__(Self,API=None):
        super(MyStreamListener,Self).__init__()
        Self.Num_Tweets=0
        Self.File=open("Tweet.txt","w")
    def on_status(Self,Status):
        Tweet=Status._json
        Self.File.write(json.dumps(Tweet)+ '\n')
        Tweet_List.append(Status)
        Self.Num_Tweets+=1
        if Self.Num_Tweets<250:
            return True
        else:
            return False
        Self.File.close()

In [ ]:
#CREATING STREAM OBJECT & AUTHENTICATE
L = MyStreamListener()
stream = tweepy.Stream(authenticate,L)

#FOR FILTERING THE TWEETS
stream.filter(track=['IoT','EdgeComputing','MobileIoT','CloudPlatform'])

In [ ]:
Data_Path='Tweet.txt'
Tweets_Data=[]
Tweets_File=open(Data_Path,"r")

#READING TWEETS AND STORING THEM
for Line in Tweets_File:
    Tweet=json.loads(Line)
    Tweets_Data.append(Tweet)
Tweets_File.close()
print((Tweets_Data[0]))

posts = []
for Key in Tweets_Data:
  posts.append(Key['text'])

posts

In [ ]:
#EXTRACTING 100 TWEETS FROM TWITTER USER
posts = T_API.user_timeline(screen_name="elonmusk", count=100, lang="en", tweet_mode="extended")

#PRINTING THE LAST 5 TWEETS FROM THE ACCOUNT
print("SHOWING 5 TWEETS: \n")
i=1
for tweet in posts[0:5]:
  print(str(i)+")", tweet.full_text, "\n")
  i+=1

In [ ]:
#CREATING DATAFRAME WITH COLUMN CALLED TWEETS
DF = pd.DataFrame([Tweet for Tweet in posts], columns=['Tweets'])

#SHOW THE FIRST 5 ROWS OF DATA
DF.head()

In [ ]:
#CLEANING THE TEXT

#CREATING A FUNCTION TO CLEAN THE TWEETS
def cleanText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) #REMOVES @ TAGS
  text = re.sub(r'#', '', text) #REMOVES # SYMBOL
  text = re.sub(r'RT[\s]+', '', text) #REMOVES RT(RETWEETS)
  text = re.sub(r'https?:\/\/\S+', '', text) #REMOVES HYPERLINK

  return text

#CLEANING THE TEXT
DF['Tweets'] = DF['Tweets'].apply(cleanText)

#SHOWING CLEANED TEXT
DF

In [ ]:
#CREATING A FUNCTION TO GET SUBJECTIVITY
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

#CREATING A FUNCTION TO GET POLARITY
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#CREATE NEW COLUMNS
DF['Subjectivity'] = DF['Tweets'].apply(getSubjectivity)
DF['Polarity'] = DF['Tweets'].apply(getPolarity)

#SHOWING DATAFRAME WITH NEW COLUMNS
DF

In [ ]:
#PLOTTING WORD CLOUD TO GET AN IDEA
allWords = ' '.join([twts for twts in DF['Tweets']])
wordCloud = WordCloud(collocations = False, background_color = 'white').generate(allWords)

Image = plt.imshow(wordCloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
#FUNCTION TO COMPUTE THE NEGETIVE NEUTRAL AND POSITIVE ANALYSIS
def getAnalysis(score):
  if score>0:
    return "POSITIVE"
  elif score==0:
    return "NEUTRAL"
  else:
    return "NEGATIVE"

DF['Analysis'] = DF['Polarity'].apply(getAnalysis)

#SHOW DATAFRAME
DF

In [ ]:
#PRINTING ALL POSITIVE TWEETS
j=1
sortedDF = DF.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == "POSITIVE"):
    print(str(j)+')', sortedDF['Tweets'][i], "\n")
    j=j+1;

In [ ]:
#PRINTING ALL NEGATIVE TWEETS
j=1
sortedDF = DF.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == "NEGATIVE"):
    print(str(j)+')', sortedDF['Tweets'][i], sortedDF['Polarity'][i], "\n")
    j=j+1;

In [ ]:
#PRINTING ALL NEUTRAL TWEETS
j=1
sortedDF = DF.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
  if (sortedDF['Analysis'][i] == "NEUTRAL"):
    print(str(j)+')', sortedDF['Tweets'][i], "\n")
    j=j+1;

In [ ]:
#PLOTTING THE POLARITY AND SUBJECTIVITY
plt.figure(figsize=(8,6))
for i in range(0, DF.shape[0]):
  plt.scatter(DF['Polarity'][i], DF['Subjectivity'][i], color='Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
#GET THE PERCENTAGE OF POSITIVE TWEETS
posTweets = DF[DF.Analysis == 'POSITIVE']
posTweets = posTweets['Tweets']

round((posTweets.shape[0] / DF.shape[0])*100, 1)

In [ ]:
#GET THE PERCENTAGE OF NEGATIVE TWEETS
negTweets = DF[DF.Analysis == 'NEGATIVE']
negTweets = negTweets['Tweets']

round((negTweets.shape[0] / DF.shape[0])*100, 1)

In [ ]:
#GET THE PERCENTAGE OF NEUTRAL TWEETS
neuTweets = DF[DF.Analysis == 'NEUTRAL']
neuTweets = neuTweets['Tweets']

round((neuTweets.shape[0] / DF.shape[0])*100, 1)

In [ ]:
#SHOWING PERCENTAGES ON GRAPH
DF['Analysis'].value_counts()

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
DF['Analysis'].value_counts().plot(kind='bar')
plt.show()